# Visualizing a Topic Model using pyLDAVis

For this project, we try to see if we can conduct some topic modelling using sklearn's in-built LDA and pyLDAVis to visualise the topic models.

First, we import the libraries needed, including:

- **sklearn** library for running LDA on the comments
- **pyLDAVis** for visualisation
- **MYSQLDb** library for connecting to the database and extract the comments
- Stanford's **nltk** (natural language toolkit) for text processing
- **pandas** for its dataframe

Note that this notebook runs on Python 2.7 due to pyLDAVis not being able to run on Python 3.

In [1]:
# supress warnings on sklearn LDA
import warnings
warnings.filterwarnings("ignore")

from __future__ import print_function

# import pyLDAVis
import pyLDAvis
import pyLDAvis.sklearn
pyLDAvis.enable_notebook()

# import MySQLdb to use the connection later
import MySQLdb

# import re for regex
import re

# import sklearn vectorizer and LDA library
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation

# import pandas for dataframe
import pandas as pd

from nltk.corpus import stopwords
import nltk
nltk.download('stopwords');

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Specify the connection information to the MySQL database and then connect to the database using pandas, retrieving the data as a dataframe. We only need the comments column for the topic modelling.

In [2]:
connec = MySQLdb.connect(host="localhost",    # your host, usually localhost
                     user="root",         # your username
                     passwd="user1991",  # your password
                     db="tia_test")        # name of the data base

comments_df = pd.read_sql('select * from comments;', con=connec)
comments_doc = comments_df['comment']

Next we look at how we can clean up the text in the comments. Some approaches are as follows:

- setting all words to lowercase
- removing all punctuations
- removing stopwords based on stopword list
- removing all tokens with only numbers
- strip leading and trailing spaces
- customise some terms (will see later)

In [3]:
# custom stopword list based on understanding of the dataset
#stop_word_list = ["salary", "cum", "up", "to", "mnc", "mncs", "job", "id", "etc", "bonus", "per", "month", "aws", "days",
#                 "immediate", "position", "positions", "temp", "temporary", "perm", "permanant", "urgent", "sea", 
#                  "singapore", "apac", "asia", "pacific", "ot", "weekend", "weekday"]
stop_word_list = ["the", "be", "to", "of", "and", "a", "in", "that", "have", "i", "it", "for", "not", "on", "with", "he", "as", "you", "do", "at",
                  "this", "but", "his", "by", "from", "they", "we", "say", "her", "she", "or", "an", "will", "my", "one", "all", "would", "there",
                  "their", "what", "so", "up", "out", "if", "about", "who", "get", "which", "go", "me", "when", "make", "can", "like", "time", "no",
                  "just", "him", "know", "take", "people", "into", "year", "your", "good", "some", "could", "them", "see", "other", "than", "then",
                  "now", "look", "only", "come", "its", "over", "think", "also", "back", "after", "use", "two", "how", "our", "work", "first", "well",
                  "way", "even", "new", "want", "because", "any", "these", "give", "day", "most", "us", 'https', 'www', 'crystal',
                  'etc', 'is', 'youre', 'were', 'im', 'hes', 'shes', 'theyre', 'are', 'yes', 'hi', 'isnt', 'wasnt', 'hey', 'co']

stops = list(stopwords.words('english'))

def treat_text(text):
    # decode text to work with special characters
    text = text.decode('utf8')
    
    # lower case the words
    text = text.lower()
    
    # substitution for consistency for words
    text = re.sub("e-commerce", "ecommerce", text)
    text = re.sub("go-jek", "gojek", text)
    
    # custom regexes based on text   
    # remove all symbols except letters numbers and blank spaces
    text = re.sub(r'[^A-Za-z0-9\s]+', ' ', text)
    
    text = re.sub(r'\b[0-9]+\b\s*', '', text)
    # remove stopwords based on nltk stop list and some custom words from looking at the dataset
    text = ' '.join([word for word in text.split() if word not in stop_word_list and word not in stops])
    
    # substitution for consistency for words
    text = re.sub("tia", "tech-in-asia", text)
    text = re.sub("tech in asia", "tech-in-asia", text)
    text = re.sub("data sets", "datasets", text)
    text = re.sub(" ai ", " artificial-intelligence ", text)
    text = re.sub(" ml ", " machine-learning ", text)
    text = re.sub("data science", "data-science", text)
    text = re.sub("data sciences", "data-science", text)
    text = re.sub("se asia", "se-asia", text)
    text = re.sub("startups", "startup", text)
    text = re.sub("southeast asia", "se-asia", text)
    text = re.sub("south-east asia", "se-asia", text)
    text = re.sub("facebooks", "facebook", text)
    text = re.sub("icos", "ico", text)
    
    # compress all more than 1 blank spaces into a single blank space and finally strip the trailing and leading blank spaces
    text = re.sub("\s\s+" , " ", text).strip()
    
    return text

In [4]:
def check_cleaning(text):
    print('Before:', text.lower())                   # title before cleaning
    print('After:', treat_text(text))        # title after cleaning

In [5]:
check_cleaning(comments_doc[172])
print()
check_cleaning(comments_doc[555])
print()
check_cleaning(comments_doc[722])
print()
check_cleaning(comments_doc[922])
print()

Before: i would actually say that data science is not mission-critical for most e-commerce/marketplace/etc startup companies. data science has the most predictive power and application when you have large data sets, which frankly, most startups will not have.
in the early years of go-jek, the role and vision of data started very early  we started small, and ensured that we were exceptionally involved in the product development from the get-go. that is, just ensure that there is a working feedback loop between data teams and other parts of the company, such as finance, product owners, and marketing, and it should happen as early as possible. because the bigger a company is, the harder it is to integrate effectively, or at all. if you have proper data sets, the data science can come later. solve the easy problems first 
make sure that data is structured properly first. having vast amounts of unstructured data is a sign that the bi team and a product development team were not coordinating

In [6]:
# remove duplicates and apply text cleaning on entire dataframe
comments_parsed = (comments_doc).apply(treat_text)

In [7]:
tf_vectorizer = CountVectorizer(analyzer='word',
                                max_df = 0.5, 
                                min_df = 10)

dtm_tf = tf_vectorizer.fit_transform(comments_parsed)
print(dtm_tf.shape)

(1437, 513)


In [8]:
lda_tf = LatentDirichletAllocation(n_topics=20, 
                                   random_state=0,
                                   max_iter=100)
lda_tf.fit(dtm_tf)

C:\Users\User\Anaconda2\envs\gl-env\lib\site-packages\sklearn\decomposition\online_lda.py:294: DeprecationWarning: n_topics has been renamed to n_components in version 0.19 and will be removed in 0.21
  DeprecationWarning)
C:\Users\User\Anaconda2\envs\gl-env\lib\site-packages\sklearn\decomposition\online_lda.py:536: DeprecationWarning: The default value for 'learning_method' will be changed from 'online' to 'batch' in the release 0.20. This warning was introduced in 0.18.
  DeprecationWarning)


LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
             evaluate_every=-1, learning_decay=0.7, learning_method=None,
             learning_offset=10.0, max_doc_update_iter=100, max_iter=100,
             mean_change_tol=0.001, n_components=10, n_jobs=1, n_topics=20,
             perp_tol=0.1, random_state=0, topic_word_prior=None,
             total_samples=1000000.0, verbose=0)

In [9]:
pyLDAvis.sklearn.prepare(lda_tf, dtm_tf, tf_vectorizer)

C:\Users\User\Anaconda2\envs\gl-env\lib\site-packages\pyLDAvis\_prepare.py:387: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  topic_term_dists = topic_term_dists.ix[topic_order]


PreparedData(topic_coordinates=            Freq  cluster  topics         x         y
topic                                                
7      13.732861        1       1 -0.155844 -0.124072
10      8.236688        1       2 -0.143956 -0.031038
16      7.490534        1       3 -0.114311  0.014437
2       7.063773        1       4  0.066919 -0.187759
12      6.641333        1       5  0.004416  0.184264
17      6.568884        1       6 -0.116510 -0.201645
13      6.152185        1       7 -0.155471  0.085690
15      6.075174        1       8 -0.058256 -0.023648
4       4.864339        1       9  0.020723  0.096230
6       4.765542        1      10 -0.094720 -0.044953
5       4.557570        1      11 -0.137101  0.092112
11      4.262351        1      12  0.070518 -0.075220
19      4.237344        1      13  0.024994  0.206753
18      3.799340        1      14  0.043341  0.146408
14      3.620933        1      15  0.151444 -0.035468
0       2.293637        1      16  0.218175 -0.068149
3       2.278279        1      17  0.115480  0.060474
1       1.332397        1      18  0.142471 -0.035845
8       1.241290        1      19  0.059588 -0.011738
9       0.785548        1      20  0.058100 -0.046834, topic_info=     Category        Freq          Term       Total  loglift  logprob
term                                                                 
457   Default  183.000000        thanks  183.000000  30.0000  30.0000
191   Default  172.000000         great  172.000000  29.0000  29.0000
28    Default  133.000000       article  133.000000  28.0000  28.0000
448   Default  122.000000          tech  122.000000  27.0000  27.0000
236   Default   71.000000   interesting   71.000000  26.0000  26.0000
215   Default   46.000000          idea   46.000000  25.0000  25.0000
87    Default   80.000000     companies   80.000000  24.0000  24.0000
196   Default   36.000000          guys   36.000000  23.0000  23.0000
308   Default  102.000000          much  102.000000  22.0000  22.0000
114   Default  112.000000          data  112.000000  21.0000  21.0000
348   Default   52.000000         point   52.000000  20.0000  20.0000
31    Default   61.000000          asia   61.000000  19.0000  19.0000
433   Default  134.000000       startup  134.000000  18.0000  18.0000
306   Default   79.000000         money   79.000000  17.0000  17.0000
382   Default   81.000000        really   81.000000  16.0000  16.0000
312   Default   34.000000         needs   34.000000  15.0000  15.0000
287   Default   93.000000        market   93.000000  14.0000  14.0000
276   Default   56.000000           lot   56.000000  13.0000  13.0000
456   Default   72.000000         thank   72.000000  12.0000  12.0000
96    Default   51.000000       content   51.000000  11.0000  11.0000
202   Default   30.000000         hello   30.000000  10.0000  10.0000
220   Default   83.000000            in   83.000000   9.0000   9.0000
379   Default   54.000000          read   54.000000   8.0000   8.0000
311   Default   81.000000          need   81.000000   7.0000   7.0000
77    Default   61.000000         china   61.000000   6.0000   6.0000
367   Default   41.000000  propertyguru   41.000000   5.0000   5.0000
340   Default   28.000000         phone   28.000000   4.0000   4.0000
469   Default   33.000000           top   33.000000   3.0000   3.0000
408   Default   44.000000       sharing   44.000000   2.0000   2.0000
414   Default   57.000000     singapore   57.000000   1.0000   1.0000
...       ...         ...           ...         ...      ...      ...
45    Topic20    0.064881            bi   10.129876  -0.2041  -7.3342
415   Topic20    0.064881        single   10.163477  -0.2075  -7.3342
219   Topic20    0.064881       improve   10.166264  -0.2077  -7.3342
221   Topic20    0.064881     including   10.209593  -0.2120  -7.3342
7     Topic20    0.064881        advice   10.298557  -0.2207  -7.3342
251   Topic20    0.064881        korean   15.120375  -0.6047  -7.3342
457   Topic20    0.064881        thanks